# Nightlights Generator

---
*Christian Johannes Meyer ([christian.meyer@eui.eu](christian.meyer@eui.eu)) and Espen Beer Prydz ([eprydz@worldbank.org](eprydz@worldbank.org))*
 
Sept 2, 2016

---

This notebook uses [Google Earth Engine](https://earthengine.google.com) to calculate total nightlights over all World Bank countries and economies, using shapefiles of the World Bank map unit. Learn more about World Bank country classifications [here](https://datahelpdesk.worldbank.org/knowledgebase/articles/906519). The resulting CSV file to saved in Google Drive. A second Jupyter notebook is used to reshape the CSV file and add basic country indicators. 

We use data on stable lights of the [NOAA DMSP-OLS Nighttime Lights Time Series](https://code.earthengine.google.com/dataset/NOAA/DMSP-OLS/NIGHTTIME_LIGHTS). 

Before running this notebook, you need to install the Google Earth Engine Python API. The package lives on [Github](https://github.com/google/earthengine-api). Google provides detailed instructions on [how to install the client library](https://developers.google.com/earth-engine/python_install#installing-the-client-library) and [how to set up your Google authentication credentials](https://developers.google.com/earth-engine/python_install#setting-up-authentication-credentials). 

---

## 1. Preliminary

In [1]:
%matplotlib inline

#### Import relevant packages

In [2]:
# Import the Earth Engine Python Package
import ee

# JSON
import json

# Pandas
from pandas import Series, DataFrame
from pandas.io.json import json_normalize
import pandas as pd

# Geopandas
import geopandas as gpd
from geopandas import GeoDataFrame

# Folium for quick visualization on Leaflet map
import folium

# Import other Python packages we need
from geojson import FeatureCollection, Feature, Polygon, Point
from shapely.geometry import shape
import os, fnmatch, sys, datetime, time, itertools, fiona
from time import sleep 
from IPython.display import Image, HTML
import re
from IPython.core.display import display, HTML

# Pandas datareader for WDI API access
from pandas_datareader import wb as wdi

#### Initialize Google Earth Engine

In [3]:
ee.Initialize()

#### Parameters

In [4]:
# Define paths to input and output files
CSV_PATH = 'output' 
GEOMETRY_PATH = 'static' 
GEOMETRY_FILE = 'wb_geo_ee.geojson' # World Bank boundaries

# Source collection or image
SOURCE = 'NOAA/DMSP-OLS/_NIGHTTIME_LIGHTS'

# Settings for EE data
# This should be changed in production
REDUCTION_SCALE = 1000

# Extraction dates
YEAR_START = 1992
YEAR_END = 2015

## 2. Import Geodata

#### Get night lights data from Google Earth Engine

In [5]:
collection = ee.ImageCollection(SOURCE)

Select only stable lights, extract relevant years, and sort for years from beginning of dataset

In [6]:
collection = collection.select('stable_lights').sort('system:time_start').filterDate(str(YEAR_START)+'-01-01', str(YEAR_END)+'-01-01')

#### Import shapefile from World Bank


Open geoJSON file as dictionary

In [7]:
wbgeojson = json.load(open(GEOMETRY_PATH + '/' + GEOMETRY_FILE))

Display inline as map to check import and coverage

In [8]:
boundarymap = folium.Map([30, 0],zoom_start=1,detect_retina=True,tiles='cartodbpositron')
folium.features.GeoJson(wbgeojson).add_to(boundarymap)
boundarymap

In [9]:
# Import all features into FeatureCollection
wbfc = ee.FeatureCollection(wbgeojson['features'])    

In [10]:
# Select a few
regions = wbfc.filterMetadata('ISO_Codes', 'starts_with', 'B');
#regions = wbfc

In [11]:
# temporaily use EE Fusiontable
#countries = ee.FeatureCollection('ft:1tdSwUL7MVpOauSgRzqVTOwdfy17KDbw-1d9omPw');
# temporarily only select South Africa, Somalia, Solomon Islands
#regions = countries.filterMetadata('Country', 'starts_with', 'So');


### 3. Compute Statistics using Earth Engine

Each band is called 'stable_lights'.  The function below replaces the band names with image name (contains satellite and year). We then map the function over the FeatureCollection with all images.

In [12]:
def feat2band(image):
    return ee.Image(image).select(['stable_lights'],[ee.Image(image).get('system:index')])

collection = collection.map(feat2band)

This function creates a stacked multiband image. We then iterate this function over each image in the FeatureCollection.

In [13]:
def iterator(current,previous):
    result = ee.Algorithms.If(previous, ee.Image(previous).addBands(current), current)
    return result
img_stack = collection.iterate(iterator,ee.Image())

Extract data interactively from EE server

In [14]:
mapped_results = ee.Image(img_stack).slice(1).reduceRegions(regions, ee.Reducer.sum(), REDUCTION_SCALE).getInfo()

In [15]:
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
   
    # Features is a list of dict with the output
    features = fc['features']

    dictarr = []
      
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        dictarr.append(attr)
       
    df = DataFrame(dictarr)
    return df

In [16]:
ee_data_df = fc2df(mapped_results)

Wrangle EE data into better shape

In [17]:
ee_data_df = pd.wide_to_long(ee_data_df, 'F', i='Name',j='feature')
ee_data_df = ee_data_df.reset_index(level=['feature'])

Extract satellite numbers and years, add source information

In [18]:
ee_data_df['feature'] = ee_data_df['feature'].astype(str)
ee_data_df['sat'] = ee_data_df['feature'].str[:2]
ee_data_df['year'] = ee_data_df['feature'].str[-4:]
ee_data_df['ee_source'] = SOURCE
ee_data_df['ee_reductionscale'] = REDUCTION_SCALE

Clean up

In [19]:
ee_data_df = ee_data_df.reset_index().set_index(['Name','year','sat'])
ee_data_df.sort_index(inplace=True)
ee_data_df = ee_data_df.rename(columns={'F': 'ee_nightlights_sum', 'ISO_Codes':'iso3c'})

Export as CSV

In [20]:
ee_data_df.to_csv(CSV_PATH + '/OpenEarth_Nightlights_byWBGeometry_Interactive.csv', sep=',', encoding='utf-8')

## 4. Get Basic Indicators and Metadata from WDI


Extract country metadata from WDI

In [21]:
wdi_meta_df = wdi.get_countries()
wdi_meta_df.rename(columns = {'name':'country'}, inplace = True)

Query a few indicators from WDI

In [22]:
wdi_indicators = ['SP.POP.TOTL', 'NY.GDP.PCAP.CD', 'NY.GNP.PCAP.CD', 'NY.GNP.PCAP.PP.CD','NY.GDP.PCAP.KD','NY.GDP.PCAP.PP.KD']
wdi_data_df = wdi.download(indicator=wdi_indicators, country='all', start=YEAR_START, end=YEAR_END).dropna().reset_index()

Merge data extracted from WDI API with WBI metadata

In [23]:
wdi_df = pd.merge(wdi_data_df,wdi_meta_df, how='outer', on='country')

#### Group EE data
We first need to aggregate night lights data from EE by iso3c, since some countries have several geometries in the WB GeoJSON file. We sum up lights by country.

In [24]:
ee_df_grouped = ee_data_df.reset_index().set_index(['iso3c','Names','year','sat'])
ee_df_grouped.sort_index(inplace=True)
ee_df_grouped = ee_df_grouped.sum(level=['iso3c','year','sat'])[['ee_nightlights_sum']].reset_index()

#### Merge grouped EE data with WDI data
For illustrative purposes, we use the intersection of keys from both frames (as opposed to keys from only WDI, only from EE, or the union).

In [25]:
df = pd.merge(wdi_df, ee_data_df.reset_index(), how='inner', on=['iso3c','year']).set_index(['Name','year','sat'])
df.sort_index(inplace=True)

Export to CSV

In [26]:
df.to_csv(CSV_PATH + '/OpenEarth_Nightlights_byWBiso3c_Interactive.csv', sep=',', encoding='utf-8')